In [1]:
def spline_cubico_natural(xs, ys):
    n = len(xs) - 1  # número de intervalos

    # Paso 1: Calcular h[i]
    h = [xs[i+1] - xs[i] for i in range(n)]

    # Paso 2: Calcular alpha[i]
    alpha = [0.0] * (n + 1)
    for i in range(1, n):
        alpha[i] = (3/h[i]) * (ys[i+1] - ys[i]) - (3/h[i-1]) * (ys[i] - ys[i-1])

    # Paso 3: Resolver sistema tridiagonal para c[i]
    l = [1.0] + [0.0] * n
    mu = [0.0] * (n + 1)
    z = [0.0] * (n + 1)
    for i in range(1, n):
        l[i] = 2 * (xs[i+1] - xs[i-1]) - h[i-1] * mu[i-1]
        mu[i] = h[i] / l[i]
        z[i] = (alpha[i] - h[i-1]*z[i-1]) / l[i]
    l[n] = 1.0
    z[n] = 0.0
    c = [0.0] * (n + 1)

    # Paso hacia atrás para calcular c[i]
    for j in reversed(range(n)):
        c[j] = z[j] - mu[j] * c[j+1]

    # Paso 4: Calcular b[i], d[i], a[i]
    b = [0.0] * n
    d = [0.0] * n
    a = ys[:-1]  # a[i] = y[i]
    for i in range(n):
        b[i] = (ys[i+1] - ys[i])/h[i] - h[i]*(2*c[i] + c[i+1])/3
        d[i] = (c[i+1] - c[i]) / (3*h[i])

    # Salida: lista de coeficientes (a, b, c, d) para cada tramo
    coeficientes = []
    for i in range(n):
        coef = (a[i], b[i], c[i], d[i], xs[i])
        coeficientes.append(coef)
    
    return coeficientes


In [2]:
# Puntos de ejemplo
xs = [0, 1, 2]
ys = [1, 5, 3]

# Calcular los splines
splines = spline_cubico_natural(xs, ys)

# Mostrar resultados
for i, (a, b, c, d, x0) in enumerate(splines):
    print(f"Spline {i}: S(x) = {a:.2f} + {b:.2f}(x - {x0}) + {c:.2f}(x - {x0})^2 + {d:.2f}(x - {x0})^3")


Spline 0: S(x) = 1.00 + 5.50(x - 0) + 0.00(x - 0)^2 + -1.50(x - 0)^3
Spline 1: S(x) = 5.00 + 1.00(x - 1) + -4.50(x - 1)^2 + 1.50(x - 1)^3


In [3]:
def spline_cubico_clamped_2p(x0, y0, x1, y1, fp0, fpn):
    h = x1 - x0
    a = y0
    b = fp0

    delta_y = y1 - y0 - b * h
    delta_fp = fpn - b

    d = (delta_fp - (2 * delta_y / h)) / (h ** 2)
    c = (delta_y - d * h ** 3) / (h ** 2)

    return a, b, c, d, x0

# Datos del problema
x0, y0 = -1, 1
x1, y1 = 1, 3
fp0 = 1
fpn = 2

# Calcular spline
a, b, c, d, base = spline_cubico_clamped_2p(x0, y0, x1, y1, fp0, fpn)

# Mostrar resultado
print(f"S(x) = {a:.2f} + {b:.2f}(x - ({base})) + {c:.2f}(x - ({base}))^2 + {d:.2f}(x - ({base}))^3")


S(x) = 1.00 + 1.00(x - (-1)) + -0.50(x - (-1))^2 + 0.25(x - (-1))^3
